In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:

spark = SparkSession.builder \
    .appName("ecommerce-etl") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33,com.clickhouse:clickhouse-jdbc:0.4.6") \
    .getOrCreate()


:: loading settings :: url = jar:file:/home/airflow/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/airflow/.ivy2.5.2/cache
The jars for the packages stored in: /home/airflow/.ivy2.5.2/jars
mysql#mysql-connector-java added as a dependency
com.clickhouse#clickhouse-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-35c2570a-304d-496d-94b3-e73b3876f474;1.0
	confs: [default]


mysql#mysql-connector-java;8.0.33 is relocated to com.mysql#mysql-connector-j;8.0.33. Please update your dependencies.
	found mysql#mysql-connector-java;8.0.33 in central


	found com.mysql#mysql-connector-j;8.0.33 in central


	found com.google.protobuf#protobuf-java;3.21.9 in central


	found com.clickhouse#clickhouse-jdbc;0.4.6 in central
downloading https://repo1.maven.org/maven2/com/clickhouse/clickhouse-jdbc/0.4.6/clickhouse-jdbc-0.4.6.jar ...


	[SUCCESSFUL ] com.clickhouse#clickhouse-jdbc;0.4.6!clickhouse-jdbc.jar (1096ms)
downloading https://repo1.maven.org/maven2/com/mysql/mysql-connector-j/8.0.33/mysql-connector-j-8.0.33.jar ...


	[SUCCESSFUL ] com.mysql#mysql-connector-j;8.0.33!mysql-connector-j.jar (1416ms)
downloading https://repo1.maven.org/maven2/com/google/protobuf/protobuf-java/3.21.9/protobuf-java-3.21.9.jar ...


	[SUCCESSFUL ] com.google.protobuf#protobuf-java;3.21.9!protobuf-java.jar(bundle) (1005ms)
:: resolution report :: resolve 4392ms :: artifacts dl 3527ms
	:: modules in use:
	com.clickhouse#clickhouse-jdbc;0.4.6 from central in [default]
	com.google.protobuf#protobuf-java;3.21.9 from central in [default]
	com.mysql#mysql-connector-j;8.0.33 from central in [default]
	mysql#mysql-connector-java;8.0.33 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   4   |   4   |   4   |   0   ||   3   |   3   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-35c2570a-304d-496d-94b3-e73b3876f474
	confs: [default]
	3 artifacts copied, 0 already retrie

25/07/21 19:44:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# إعداد الاتصال بقاعدة بيانات MySQL
mysql_url = "jdbc:mysql://mysql:3306/mydb"
mysql_properties = {
    "user": "myuser",
    "password": "mypassword",
    "driver": "com.mysql.cj.jdbc.Driver"
}


In [4]:
customers = spark.read.jdbc(mysql_url, "customers", properties=mysql_properties)
products = spark.read.jdbc(mysql_url, "products", properties=mysql_properties)
product_category_translation = spark.read.jdbc(mysql_url, "product_category_translation", properties=mysql_properties)
orders = spark.read.jdbc(mysql_url, "orders", properties=mysql_properties)
order_items = spark.read.jdbc(mysql_url, "order_items", properties=mysql_properties)
order_reviews = spark.read.jdbc(mysql_url, "order_reviews", properties=mysql_properties)
order_payments = spark.read.jdbc(mysql_url, "order_payments", properties=mysql_properties)
sellers = spark.read.jdbc(mysql_url, "sellers", properties=mysql_properties)
geolocation = spark.read.jdbc(mysql_url, "geolocation", properties=mysql_properties)

In [5]:
orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_carrier_date: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_estimated_delivery_date: string (nullable = true)



In [6]:
#df = customers

In [7]:
#for i in df.columns :
#    print(f"=================================== {i} ============================== ")
#    df.select(
#        sum(when(col(i).isNull(), 1).otherwise(0)).alias("null_count")
#    ).show()

In [8]:
#for i in df.columns:
#    if isinstance(df.schema[i].dataType, StringType):  # Check if column is string type
#        print(f"================================= {i} ============================================")
#        df.groupBy(i).agg(count("*").alias("value_count")).orderBy(col("value_count")).show(n=2000,truncate=False)

In [9]:
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================

In [10]:
#======================================================= customers ===========================================
customers_clean = customers.dropDuplicates(["customer_id"])
customers_clean = customers_clean.filter(length("customer_zip_code_prefix") == 5)
customers_clean = customers_clean.filter(length("customer_state") == 2)


In [11]:
corrections = {
    "sao jorge d oeste": "sao jorge do oeste",
    "estrela d oeste": "estrela do oeste", 
    "palma d oeste": "palma do oeste",
    "sao jorge d'oeste": "sao jorge do oeste",
    "estrela d'oeste": "estrela do oeste",
    "palma d'oeste": "palma do oeste",
    "alvorada d'oeste": "alvorada do oeste",
    "diamante d'oeste": "diamante do oeste",
    "rancho alegre d'oeste": "rancho alegre do oeste",
    "bandeirantes d'oeste": "bandeirantes do oeste",
    "santa clara d'oeste": "santa clara do oeste",
    "santa rita d'oeste": "santa rita do oeste",
    "aparecida d'oeste": "aparecida do oeste",
    "palmeira d oeste": "palmeira do oeste",
    "palmeira d'oeste": "palmeira do oeste" ,
    'sao paulo - sp': 'sao paulo',
    'sao paulo sp': 'sao paulo',
    'sp': 'sao paulo',
    'sao  paulo': 'sao paulo',
    'sao pauo': 'sao paulo',
    'sao paulop': 'sao paulo',
    'são paulo': 'sao paulo',
    'sbc': 'sao bernardo do campo',
    'sbc/sp': 'sao bernardo do campo',
    'riberao preto': 'ribeirao preto',
    'robeirao preto': 'ribeirao preto',
    'ribeirao pretp': 'ribeirao preto',
    'sando andre': 'santo andre',
    'ao bernardo do campo': 'sao bernardo do campo',
    'garulhos': 'guarulhos',
    'brasilia df': 'brasilia',
    'rio de janeiro / rio de janeiro': 'rio de janeiro',
    'rio de janeiro, rio de janeiro, brasil': 'rio de janeiro',
    'mogi das cruses': 'mogi das cruzes',
    'mogi das cruzes / sp': 'mogi das cruzes',
    'santo andre/sao paulo': 'santo andre',
    "santa barbara d´oeste": "santa barbara d'oeste",
    "santa barbara d oeste": "santa barbara d'oeste",
    'belo horizont': 'belo horizonte',
    'cascavael': 'cascavel',
    'floranopolis': 'florianopolis',
}

In [12]:
# الخطوة 1: تنظيف أساسي (يتم تطبيقه مباشرة على العمود الأصلي)
customers_clean = customers_clean.withColumn(
    "customer_city",
    lower(trim(regexp_replace(col("customer_city"), "[^a-zA-ZÀ-ÿ ]", "")))
)

# الخطوة 2: تصحيح التهجئات الشائعة
customers_clean = customers_clean.withColumn(
    "customer_city",
    regexp_replace(col("customer_city"), "\\bd['\\s]o\\b", " do ")
)


for wrong, correct in corrections.items():
    customers_clean = customers_clean.withColumn(
        "customer_city",
        when(col("customer_city") == wrong, correct).otherwise(col("customer_city"))
)


In [13]:
#print(f"================================= customer_state ============================================")
#customers.groupBy("customer_city").agg(count("*").alias("value_count")).orderBy(col("value_count")).show(n=20000,truncate=False)

In [14]:
#print(f"================================= customer_state ============================================")
#customers_clean.groupBy("customer_city").agg(count("*").alias("value_count")).orderBy(col("value_count")).show(n=20000,truncate=False)

In [15]:
print("================================= Distinct customer_city count ============================================")
distinct_count = customers_clean.select("customer_city").distinct().count()
print(f"عدد القيم المختلفة في customer_city هو: {distinct_count}")

================================= Distinct customer_city count ============================================


عدد القيم المختلفة في customer_city هو: 4078


In [16]:
print("================================= Distinct customer_city count ============================================")
distinct_count = customers.select("customer_city").distinct().count()
print(f"عدد القيم المختلفة في customer_city هو: {distinct_count}")

================================= Distinct customer_city count ============================================


عدد القيم المختلفة في customer_city هو: 4119


In [17]:
customers_clean.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: long (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [18]:
#======================================================= products ===========================================
products_clean = products.filter(products.product_category_name.isNotNull())
products_clean = products_clean.dropna(subset=[
    "product_weight_g", "product_length_cm", "product_height_cm", "product_width_cm"
])
products_clean = products_clean \
    .withColumn("product_name_lenght", col("product_name_lenght").cast("int")) \
    .withColumn("product_description_lenght", col("product_description_lenght").cast("int")) \
    .withColumn("product_photos_qty", col("product_photos_qty").cast("int")) \
    .withColumn("product_weight_g", col("product_weight_g").cast("int")) \
    .withColumn("product_length_cm", col("product_length_cm").cast("int")) \
    .withColumn("product_height_cm", col("product_height_cm").cast("int")) \
    .withColumn("product_width_cm", col("product_width_cm").cast("int"))

products_clean = products_clean.withColumn(
    "product_volume_cm3",
    expr("product_length_cm * product_height_cm * product_width_cm")
)
products_clean = products_clean.withColumn(
    "product_category_name", trim(lower("product_category_name"))
)


In [19]:
products_clean.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)
 |-- product_volume_cm3: integer (nullable = true)



In [20]:
#======================================================= product_category_translation ===========================================
product_category_translation_clean = product_category_translation.withColumn(
    "product_category_name", trim(lower("product_category_name"))
)


In [21]:
#=======================================================  product ===========================================
products_enriched = products_clean.join(
    product_category_translation_clean,
    on="product_category_name",
    how="left"
)

In [22]:
#======================================================= orders ===========================================

orders_clean = orders.filter(
    ~col("order_status").isin("created", "approved")
)

orders_clean = orders_clean \
    .withColumn("order_purchase_timestamp", to_timestamp("order_purchase_timestamp")) \
    .withColumn("order_approved_at", to_timestamp("order_approved_at")) \
    .withColumn("order_delivered_carrier_date", to_timestamp("order_delivered_carrier_date")) \
    .withColumn("order_delivered_customer_date", to_timestamp("order_delivered_customer_date")) \
    .withColumn("order_estimated_delivery_date", to_timestamp("order_estimated_delivery_date"))

orders_clean = orders_clean.withColumn(
    "shipping_days",
    datediff("order_delivered_customer_date", "order_purchase_timestamp")
)
orders_clean = orders_clean.withColumn(
    "delivery_delay",
    datediff("order_delivered_customer_date", "order_estimated_delivery_date")
)

In [23]:
orders_clean.select("order_id").distinct().count() == orders_clean.count()

True

In [24]:
#======================================================= order_items ===========================================
#order_items_clean = order_items_clean.filter((col("price") <= 0) | (col("freight_value") < 0))
order_items_clean = order_items.withColumn(
    "shipping_limit_date", to_timestamp("shipping_limit_date")
)

# نحتفظ بالصفوف اللي سعرها أكبر من صفر
order_items_clean = order_items_clean.filter((col("price") > 0) & (col("freight_value") >= 0))

# نضيف total_cost
order_items_clean = order_items_clean.withColumn(
    "total_cost", expr("price + freight_value")
)

In [25]:
#order_items.show()

In [26]:
#======================================================= order_reviews ===========================================
order_reviews_clean = order_reviews \
    .withColumn("review_creation_date", to_timestamp("review_creation_date")) \
    .withColumn("review_answer_timestamp", to_timestamp("review_answer_timestamp"))

order_reviews_clean = order_reviews_clean \
    .withColumn("has_message", when(order_reviews_clean.review_comment_message.isNull(), 0).otherwise(1)) \
    .withColumn("has_title", when(order_reviews_clean.review_comment_title.isNull(), 0).otherwise(1))

order_reviews_clean = order_reviews_clean \
    .withColumn("clean_comment_message", trim(lower(regexp_replace("review_comment_message", "[^a-zA-Z0-9çãáéíóúâêôõüàèìòùÇÃÁÉÍÓÚÂÊÔÕÜÀÈÌÒÙ ]", "")))) \
    .withColumn("clean_comment_title", trim(lower(regexp_replace("review_comment_title", "[^a-zA-Z0-9çãáéíóúâêôõüàèìòùÇÃÁÉÍÓÚÂÊÔÕÜÀÈÌÒÙ ]", ""))))

order_reviews_clean = order_reviews_clean.withColumn(
    "review_sentiment",
    when(col("review_score") >= 4, "positive")
    .when(col("review_score") == 3, "neutral")
    .otherwise("negative")
)

In [27]:
#======================================================= order_payments ===========================================
order_payments_clean = order_payments.groupBy("order_id").agg(
    sum("payment_value").alias("total_payment"),
    sum("payment_installments").alias("total_installments"),
    count_distinct("payment_type").alias("payment_methods_count"),
    first("payment_type").alias("main_payment_type")
)

In [28]:
#=======================================================  order ===========================================
orders_items_joined = orders_clean.join(
    order_items_clean,
    on="order_id",
    how="left"
)
orders_items_payments_joined = orders_items_joined.join(
    order_payments_clean,
    on="order_id",
    how="left"
)
orders_full_cleaned = orders_items_payments_joined.join(
    order_reviews_clean,
    on="order_id",
    how="left"
)


In [29]:
#orders_clean.select("seller_id", "product_id","shipping_limit_date", "price","freight_value", "total_cost").show()


In [30]:
#=======================================================  sellers ===========================================
# الخطوة 1: تنظيف أساسي (يتم تطبيقه مباشرة على العمود الأصلي)
sellers_cleaned = sellers.withColumn(
    "seller_city",
    lower(trim(regexp_replace(col("seller_city"), "[^a-zA-ZÀ-ÿ ]", "")))
)

# الخطوة 2: تصحيح التهجئات الشائعة
sellers_cleaned = sellers_cleaned.withColumn(
    "seller_city",
    regexp_replace(col("seller_city"), "\\bd['\\s]o\\b", " do ")
)


for wrong, correct in corrections.items():
    sellers_cleaned = sellers_cleaned.withColumn(
        "seller_city",
        when(col("seller_city") == wrong, correct).otherwise(col("seller_city"))
)


In [31]:
#=======================================================  geolocation ===========================================
geolocation_clean = geolocation.dropna(subset=[
    "geolocation_zip_code_prefix", 
    "geolocation_lat", 
    "geolocation_lng", 
    "geolocation_city", 
    "geolocation_state"
]).dropDuplicates(["geolocation_zip_code_prefix", "geolocation_city", "geolocation_state"])


In [32]:
#tables = {
#    "customers_clean": customers_clean,
#    "products_enriched": products_enriched, 
#    "orders_full_cleaned": orders_full_cleaned,
#    "sellers_cleaned": sellers_cleaned,
#    "geolocation_clean": geolocation_clean
#}

#for name, df in tables.items():
#    print(f"\n=== Schema for {name} ===")
#    df.printSchema()

In [33]:
#================================================================================================================
#================================================================================================================
#================================================================================================================
#================================================================================================================
#================================================================================================================
#================================================================================================================
#================================================================================================================

In [34]:
dim_customers = customers_clean.select(
    "customer_id",
    "customer_unique_id",
    "customer_zip_code_prefix",
    "customer_city",
    "customer_state"
).dropDuplicates(["customer_id"])


In [35]:
dim_sellers = sellers_cleaned.select(
    "seller_id",
    "seller_zip_code_prefix",
    "seller_city",
    "seller_state"
).dropDuplicates(["seller_id"])


In [36]:
dim_products = products_enriched.select(
    "product_id",
    "product_category_name",
    "product_category_name_english",
    "product_name_lenght",
    "product_description_lenght",
    "product_photos_qty",
    "product_weight_g",
    "product_length_cm",
    "product_height_cm",
    "product_width_cm",
    "product_volume_cm3"
).dropDuplicates(["product_id"])


In [37]:
#dim_geolocation = geolocation_clean.select(
#    col("geolocation_zip_code_prefix").alias("zip_code_prefix"),
#    "geolocation_lat",
#    "geolocation_lng",
#    "geolocation_city",
#    "geolocation_state"
#).dropDuplicates(["zip_code_prefix"])


In [38]:
min_max = orders_full_cleaned.select(
    min("order_purchase_timestamp").alias("start"),
    max("order_purchase_timestamp").alias("end")
).first()

start_date = min_max["start"].date()
end_date = min_max["end"].date()

date_df = spark.sql(f"""
    SELECT explode(sequence(to_date('{start_date}'), to_date('{end_date}'), interval 1 day)) as date_key
""")

dim_date = date_df.withColumn("day", dayofmonth("date_key")) \
                  .withColumn("month", month("date_key")) \
                  .withColumn("year", year("date_key")) \
                  .withColumn("day_of_week", date_format("date_key", "E")) \
                  .withColumn("week_of_year", weekofyear("date_key")) \
                  .withColumn("day_name", date_format("date_key", "EEEE")) \
                  .withColumn("month_name", date_format("date_key", "MMMM"))

In [39]:

fact_orders = orders_full_cleaned.select(
    "order_id",
    "customer_id",
    "seller_id",
    "product_id",

    # 🔑 تواريخ على هيئة timestamps
    "order_purchase_timestamp",
    "shipping_limit_date",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",

    # 🔢 measures
    "shipping_days",
    "delivery_delay",
    "price",
    "freight_value",
    "total_cost",
    "total_payment",
    "total_installments",
    "payment_methods_count",    # أضفناه
    "main_payment_type",

    # 🔍 تقييم العميل (score + sentiment فقط)
    "review_score",
    "review_sentiment",

    # ▶️ مفاتيح التواريخ لربط dim_date
    to_date("order_purchase_timestamp").alias("purchase_date_key"),
    to_date("order_delivered_customer_date").alias("delivered_date_key"),
    to_date("order_estimated_delivery_date").alias("estimated_date_key")
)


In [40]:
#tables = {
#    "dim_customers": dim_customers,
#    "dim_sellers": dim_sellers, 
#    "dim_products": dim_products,
#    "dim_date": dim_date,
#    "fact_orders": fact_orders
#}

#for name, df in tables.items():
#    print(f"\n=== Schema for {name} ===")
#    df.printSchema()

In [41]:
#===================================================================================================================
#===================================================================================================================
#===================================================================================================================
#===================================================================================================================
#===================================================================================================================
#===================================================================================================================
#===================================================================================================================

In [42]:
clickhouse_url = "jdbc:clickhouse://clickhouse:8123/ecommerce"
clickhouse_properties = {
    "user": "default",
    "password": "123",
    "driver": "com.clickhouse.jdbc.ClickHouseDriver"
}

In [43]:
dim_customers.write.jdbc(clickhouse_url, "dim_customers", mode="append", properties=clickhouse_properties)
dim_sellers.write.jdbc(clickhouse_url, "dim_sellers", mode="append", properties=clickhouse_properties)
dim_products.write.jdbc(clickhouse_url, "dim_products", mode="append", properties=clickhouse_properties)
dim_date.write.jdbc(clickhouse_url, "dim_date", mode="append", properties=clickhouse_properties)
fact_orders.write.jdbc(clickhouse_url, "fact_orders", mode="append", properties=clickhouse_properties)


25/07/21 19:44:46 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:46 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:46 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:46 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:46 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:46 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQ

25/07/21 19:44:47 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:47 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [c2c7ef73-0c92-4e80-a8d9-bb4efabb3b98] (10 queries & 0 savepoints) is committed.
25/07/21 19:44:47 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [63f2ce61-b03b-45e1-94fa-40857af03672] (0 queries & 0 savepoints) is committed.
25/07/21 19:44:47 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:47 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [91e6e8a2-41e6-4d2f-8070-50a901c0a5a6] (10 queries & 0 savepoints) is committed.
25/07/21 19:44:47 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [984e7796-77fb-4976-9ae7-ea96fdcc97b6] (0 queries & 0 savepoints) is committed.
25/07/

25/07/21 19:44:47 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:47 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [cc7a6191-1eec-4e70-b502-eadcdab8371c] (16 queries & 0 savepoints) is committed.
25/07/21 19:44:47 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [e3b4cf56-0e85-456e-a7f5-e9ad44b4152d] (0 queries & 0 savepoints) is committed.


25/07/21 19:44:48 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:48 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:48 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [dbf2b26c-32e6-402f-b3b6-ff7c2b72a23f] (5 queries & 0 savepoints) is committed.
25/07/21 19:44:48 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [7ecb905a-1572-450a-ad30-858d2b554e2d] (0 queries & 0 savepoints) is committed.


25/07/21 19:44:49 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:49 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:49 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.


25/07/21 19:44:49 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:49 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [0fc9842a-4519-45a5-a983-0599a93ca740] (12 queries & 0 savepoints) is committed.
25/07/21 19:44:49 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [def0584f-1d65-4cc7-89b5-3f3eb73ebb84] (0 queries & 0 savepoints) is committed.
25/07/21 19:44:49 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:49 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [bf499a4a-4504-45a7-8f63-5f6b132a1845] (12 queries & 0 savepoints) is committed.
25/07/21 19:44:49 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [a5c9d853-fbed-4a1b-abe1-c1b22162252f] (0 queries & 0 savepoints) is committed.
25/07/

25/07/21 19:44:50 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:50 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:50 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [667101fe-c753-4e46-8630-b719e35416eb] (2 queries & 0 savepoints) is committed.
25/07/21 19:44:50 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [7e99a5ad-e111-4701-9c3a-d6a6a0b9e575] (0 queries & 0 savepoints) is committed.
25/07/21 19:44:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


25/07/21 19:44:53 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:53 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:53 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:53 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:53 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:53 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQ

25/07/21 19:44:53 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:53 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [c6a80ea3-10a9-448e-bb92-73ee0ac9eb13] (7 queries & 0 savepoints) is committed.
25/07/21 19:44:53 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [e263ea2a-444a-47a9-9c79-6c72e66371f4] (0 queries & 0 savepoints) is committed.
25/07/21 19:44:53 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/07/21 19:44:53 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [f6e17829-f9ff-4b38-9a37-431155ae92c0] (7 queries & 0 savepoints) is committed.
25/07/21 19:44:53 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [1dbe0f8b-8822-435a-a02a-bbc92830d623] (0 queries & 0 savepoints) is committed.
25/07/21